In [ ]:
from itertools import product

from scipy.special import sph_harm

import numpy as np
import pandas as pd

from numpwd.integrate.angular import ReducedAngularPolynomial, get_x_mesh, get_phi_mesh

pd.set_option("display.precision", 3)

In [ ]:
x, wx = get_x_mesh(10)
phi, wphi = get_phi_mesh(20)

In [ ]:
ml = 0
l = 0
theta = np.arccos(x).reshape([10, 1])
yml = sph_harm(ml, l, phi, theta)

In [ ]:
np.sum(yml.conj() * yml * wx.reshape((10, 1)) * wphi)

Using the orthnormality of spherical harmonics
$$
    \int d x_1 d \phi_1 Y_{l_a m_a}^*(x_1, \phi_1) Y_{l_b m_b}(x_1, \phi_1) = \delta_{l_a l_b} \delta_{m_a m_b}
$$
we have
$$
    \int d x_1 d \phi_1 \int d x_2 d \phi_2
    Y_{l_{a1} m_{a1}}^*(x_1, \phi_1) Y_{l_{a2} m_{a2}}(x_2, \phi_2) 
    Y_{l_{b1} m_{b1}}(x_1, \phi_1) Y^*_{l_{b2} m_{b2}}(x_2, \phi_2)
    =
    \delta_{l_{a1} l_{b1}} \delta_{m_{a1} m_{b1}}
    \delta_{l_{a2} l_{b2}} \delta_{m_{a2} m_{b2}}
$$

After projecting onto $a$ operator quantum numbers and refactoring the integral

\begin{multline}
    \frac{2 \lambda + 1}{2l_{a1} + 1}
    \sum_{m_{a1} m_{a2}}
    \left\langle l_{a1} m_{a1}, \lambda m_\lambda \vert l_{a2} m_{a2} \right\rangle 
    \int d x_1 d \phi_1 \int d x_2 d \phi_2
    \\
    \times 
    Y_{l_{a1} m_{a1}}^*(x_1, 0) Y_{l_{a2} m_{a2}}(x_2, 0)
    \exp\left( -i m_{a1} \phi_1 + i m_{a2} \phi_2  \right)
    \\
    \times 
    Y_{l_{b1} m_{b1}}(x_1, \phi_1) Y^*_{l_{b2} m_{b2}}(x_2, \phi_2)
    \\
    =
    \frac{2 \lambda + 1}{2l_{b1} + 1}
    \sum_{m_{a1} m_{a2}}
    \left\langle l_{b1} m_{b1}, \lambda m_\lambda \vert l_{b1} m_{b1} \right\rangle
    \delta_{l_{a1} l_{b1}}
    \delta_{l_{a2} l_{b2}}
\end{multline}

we find $\phi_1 = \Phi - \phi/2$, $\phi_2 = \Phi + \phi/2$

\begin{multline}
    =
    \int d x_1  d x_2 d \phi
    A_{(l_{a2} l_{a1}) \lambda m_\lambda}(x_1, x_2, \phi)
    \int d \Phi
    e^{i m_\lambda \Phi}
    Y_{l_{b1} m_{b1}}(x_1, 0) Y^*_{l_{b2} m_{b2}}(x_2, 0)
    e^{-i (m_{b2} - m_{b1}) \Phi}
    e^{-i \frac{m_{b2} + m_{b1}}{2} \phi}
    \\
    =
    \int d x_1  d x_2 d \phi
    A_{(l_{a2} l_{a1}) \lambda m_\lambda}(x_1, x_2, \phi)
    2 \pi \delta_{m_\lambda, m_{b2} - m_{b1}}
    Y_{l_{b1} m_{b1}}(x_1, 0) Y^*_{l_{b2} m_{b2}}(x_2, 0)
    e^{-i \frac{m_{b2} + m_{b1}}{2} \phi}
    \\
    =
    \int d x_1  d x_2 d \phi
    A_{(l_{a2} l_{a1}) \lambda m_\lambda}(x_1, x_2, \phi)
    O_{(l_{b2} l_{b1}) m_{\lambda} m_{b1} m_{b2}}(x_1, x_2, \phi)
\end{multline}


In [ ]:
lmax = 2

channels = []
for lo, li in product(range(lmax + 1), range(lmax + 1)):
    for la in range(abs(li - lo), li + lo + 1):
        for mla in range(-la, la + 1):
            channels.append([lo, li, la, mla])

nchannels = len(channels)

In [ ]:
nx = 10
nphi = 20

matrix = np.zeros(shape=[nchannels, nx, nx, nphi], dtype=np.complex128)

x, wx = get_x_mesh(nx)
phi, wphi = get_phi_mesh(nphi)

e_i_phi = np.exp(1j * phi).reshape([1, 1, nphi]) + matrix[0]

pphi = 0
theta = np.arccos(x)  # pylint: disable=E1111

ylmi = {}
ylmo = {}
for l in range(lmax + 1):
    for ml in range(-l, l + 1):
        # WARNING: scipy uses the opposite convention as we use in physics
        ##: e.g.: l=n, ml=m, phi=theta [0, 2pi], theta=phi [0, pi]
        ## ---> left is physics, right is scipy <---
        ## The scipy call structure is thus sph_harm(m, n, theta, phi)
        ## which means we want sph_harm(ml, l, phi, theta)
        yml = sph_harm(ml, l, pphi, theta)
        ylmo[l, ml] = yml.reshape([nx, 1, 1])
        ylmi[l, ml] = yml.reshape([1, nx, 1])

for idx, (lo, li, la, mla) in enumerate(channels):
    for mlo, mli in product(range(-lo, lo + 1), range(-li, li + 1)):
        if mli + mla != mlo:
            continue

        # Note that Y_lm have been computed for phi = 0
        ## Since exp(I mla Phi) is in spin element
        ## This term only contains (ml_i + ml_o) / 2
        matrix[idx] += (
            ylmo[lo, mlo] * ylmi[li, mli] * e_i_phi ** (-(mli + mlo) / 2) * 2 * np.pi
        )

In [ ]:
ang_poly = ReducedAngularPolynomial(x, phi, lmax, wx=wx, wphi=wphi)

In [ ]:
nca = nco = 0

np.sum(
    ang_poly.matrix[nca]
    * matrix[nco]
    * wx.reshape((nx, 1, 1))
    * wx.reshape((1, nx, 1))
    * wphi.reshape((1, 1, nphi))
)

In [ ]:
res = pd.Series(ang_poly.integrate(matrix[nco], mla=0), name="val")
res.index.names = ["lo", "li", "la", "mla"]
pd.DataFrame(res)